In [1]:
#import section

import sys
sys.path.append('modules')

import json
import time
import imaplib
import email
from bs4 import BeautifulSoup
import dbstruct
import messagestruct
import messagefuncs
import dbfuncs


In [2]:
#initialisation


with open('gmail.creds') as f:
    data = json.load(f)
    gmailServer = "imap.gmail.com"
    gmailPort   = 993    
    gmailAccount = data['account']
    gmailPassword = data['password']
    
    
with open('db.creds') as f:
    data = json.load(f)
    dbHost = data['host']
    dbPort = data['port']
    dbDatabase = data['database']
    dbUser = data['user']
    dbPassword = data['password'] 

In [3]:
#connect to gmail

mail = imaplib.IMAP4_SSL(gmailServer)
mail.login(gmailAccount,gmailPassword)

('OK', [b'fitosystems@gmail.com authenticated (Success)'])

In [4]:
#querying and defining messages for processing


shiftFromLastMessage = 9

mail.select('fitosystems.ru')
#typ, data = mail.search(None, 'FROM "info@fitosystems.ru"')
typ, data = mail.uid('search', None, 'FROM "info@fitosystems.ru"')


mail_uids = data[0]
uid_list = list(map(lambda x: x.decode("utf-8"), mail_uids.split()))


# first_email_uid = uid_list[0]
# last_email_uid = uid_list[-1 - shiftFromLastMessage]


# indexFrom = 0
# indexTo = 0

# try:
#     indexFrom = uid_list.index(dblast_email_uid) + 1
# except ValueError:
#     indexFrom = 0
    
# indexTo = len(uid_list) - 1 - shiftFromLastMessage


# print(indexFrom)
# print(indexTo)

In [5]:
#processing instances

indexFrom=0
indexTo = 11393


engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)
session = dbfuncs.dbGetSession(engine)



dbuid_list = session.query(dbstruct.Order.message_id).all() 
substractList = list(set(uid_list) - set(dbuid_list))


#for index in range(indexFrom, indexTo + 1, 1):
cnt = 0
for index in substractList:

    #messageId = uid_list[index]
    messageId = index
    typ, data = mail.uid('fetch',str(messageId),'(RFC822)') 
    rawEmail = data[0][1]
    message = email.message_from_bytes(rawEmail)
    
    
    try:
        messageDict = messagefuncs.parseMessage(messageId,message)

        print(str(messageDict.order['messageId']) + ' ' + message["Date"])


        dbfuncs.submitMessage(session,messageDict)
        
    except BaseException as ex:    
        print("!!!error - " + str(ex) + ' ' + message["Date"])
        continue


    if (cnt % 500 == 0):
        session.commit()
        print("commit")
    cnt += 1


try:
    session.commit()
    print("commit")
except BaseException as ex:
    session.rollback()
    print("rollback" + str(ex))
    

engine.dispose()

12692 Sun, 16 Dec 2018 01:28:25 +0300
commit
!!!error - invalid literal for int() with base 10: '7800,00' Fri, 23 Sep 2016 16:32:27 +0300
2771 Fri, 15 Dec 2017 22:08:28 +0300
885 Mon, 9 Apr 2018 13:35:42 +0300
!!!error - list index out of range Fri, 2 Mar 2018 23:46:00 +0300
!!!error - invalid literal for int() with base 10: '21650,00' Fri, 2 Sep 2016 18:18:13 +0300
292 Wed, 30 May 2018 00:11:13 +0300
!!!error - invalid literal for int() with base 10: '350,00' Wed, 27 Sep 2017 12:55:47 +0300
12652 Thu, 6 Dec 2018 13:48:43 +0300
629 Fri, 27 Apr 2018 10:45:25 +0300
!!!error - invalid literal for int() with base 10: '87a70121' Tue, 25 Aug 2015 13:48:02 +0300
!!!error - invalid literal for int() with base 10: '9550,00' Mon, 28 Dec 2015 14:00:39 +0300
12095 Sun, 19 Aug 2018 21:40:23 +0300
1282 Mon, 12 Mar 2018 13:57:32 +0300
3655 Wed, 25 Oct 2017 09:39:01 +0300
!!!error - invalid literal for int() with base 10: '350,00' Mon, 24 Jul 2017 16:52:36 +0300
11895 Sun, 22 Jul 2018 23:15:47 +0300
2

!!!error - invalid literal for int() with base 10: '499,00' Wed, 21 Sep 2016 16:16:28 +0300
432 Wed, 16 May 2018 09:25:36 +0300
2410 Fri, 5 Jan 2018 12:38:12 +0300
!!!error - invalid literal for int() with base 10: '3549,00' Sun, 16 Jul 2017 06:47:11 +0300
!!!error - invalid literal for int() with base 10: '№' Fri, 13 Feb 2015 09:06:31 +0300
!!!error - invalid literal for int() with base 10: '475,00' Mon, 18 Sep 2017 14:26:50 +0300
!!!error - invalid literal for int() with base 10: '590,00' Wed, 9 Aug 2017 10:42:48 +0300
3323 Thu, 16 Nov 2017 13:58:01 +0300
3592 Sun, 29 Oct 2017 14:52:41 +0300
!!!error - list index out of range Wed, 6 Dec 2017 18:24:03 +0300
!!!error - invalid literal for int() with base 10: '9999,00' Fri, 23 Sep 2016 20:40:20 +0300
!!!error - invalid literal for int() with base 10: '3990,00' Mon, 10 Jul 2017 11:48:54 +0300
!!!error - invalid literal for int() with base 10: '9225,00' Thu, 6 Oct 2016 15:03:54 +0300
!!!error - invalid literal for int() with base 10: '700

13341 Thu, 18 Apr 2019 11:50:13 +0300
!!!error - invalid literal for int() with base 10: '399,00' Sun, 23 Apr 2017 03:17:37 +0300
!!!error - invalid literal for int() with base 10: '7800,00' Thu, 31 Mar 2016 18:53:20 +0300
3406 Fri, 10 Nov 2017 16:42:28 +0300
1929 Thu, 8 Feb 2018 12:27:58 +0300
2132 Wed, 24 Jan 2018 18:49:29 +0300
!!!error - invalid literal for int() with base 10: '2518,00' Wed, 14 Jun 2017 16:00:24 +0300
!!!error - invalid literal for int() with base 10: '450,00' Wed, 12 Apr 2017 17:30:08 +0300
13515 Tue, 25 Jun 2019 22:44:30 +0300
1443 Tue, 6 Mar 2018 03:31:43 +0300
11711 Thu, 5 Jul 2018 13:56:20 +0300
2518 Wed, 27 Dec 2017 16:11:49 +0300
!!!error - invalid literal for int() with base 10: 'Заказ' Tue, 14 Jul 2015 18:34:49 +0300
13323 Sun, 14 Apr 2019 11:59:35 +0300
1561 Sat, 3 Mar 2018 14:45:58 +0300
!!!error - invalid literal for int() with base 10: '3250,00' Tue, 18 Apr 2017 23:17:01 +0300
3941 Mon, 9 Oct 2017 11:13:23 +0300
1492 Mon, 5 Mar 2018 14:05:04 +0300
!!!e

2501 Thu, 28 Dec 2017 11:28:39 +0300
!!!error - invalid literal for int() with base 10: '999,00' Wed, 27 Sep 2017 09:46:14 +0300
!!!error - list index out of range Wed, 6 Dec 2017 17:05:18 +0300
!!!error - invalid literal for int() with base 10: '6250,00' Tue, 8 Sep 2015 13:18:08 +0300
1013 Fri, 30 Mar 2018 09:52:40 +0300
!!!error - list index out of range Wed, 29 Nov 2017 09:20:16 +0300
13501 Thu, 20 Jun 2019 18:31:18 +0300
12368 Fri, 5 Oct 2018 10:14:44 +0300
1198 Sat, 17 Mar 2018 16:14:40 +0300
!!!error - invalid literal for int() with base 10: '700,00' Fri, 5 May 2017 10:38:53 +0300
!!!error - invalid literal for int() with base 10: '4650,00' Sat, 7 Nov 2015 10:37:06 +0300
13548 Thu, 11 Jul 2019 17:44:11 +0300
!!!error - invalid literal for int() with base 10: '3850,00' Wed, 16 Dec 2015 09:41:25 +0300
!!!error - invalid literal for int() with base 10: '№' Sun, 17 May 2015 18:22:40 +0300
2417 Thu, 4 Jan 2018 21:46:28 +0300
!!!error - invalid literal for int() with base 10: '700,00' 

3236 Wed, 22 Nov 2017 13:58:03 +0300
!!!error - invalid literal for int() with base 10: '1839,00' Wed, 12 Jul 2017 10:33:07 +0300
!!!error - invalid literal for int() with base 10: '925,00' Tue, 12 Sep 2017 17:35:12 +0300
11944 Tue, 31 Jul 2018 17:59:54 +0300
13166 Tue, 12 Mar 2019 14:50:38 +0300
!!!error - list index out of range Sun, 26 Nov 2017 19:05:28 +0300
!!!error - invalid literal for int() with base 10: '1499,00' Thu, 12 Jan 2017 18:02:39 +0300
!!!error - invalid literal for int() with base 10: '550,00' Sat, 8 Apr 2017 17:06:31 +0300
12048 Sun, 12 Aug 2018 23:20:03 +0300
399 Sat, 19 May 2018 16:28:27 +0300
!!!error - invalid literal for int() with base 10: '153,00' Fri, 22 Sep 2017 10:32:32 +0300
13505 Sat, 22 Jun 2019 19:47:59 +0300
497 Fri, 11 May 2018 08:48:01 +0300
!!!error - invalid literal for int() with base 10: '6222,40' Tue, 25 Oct 2016 21:37:25 +0300
3594 Sun, 29 Oct 2017 14:37:11 +0300
!!!error - invalid literal for int() with base 10: '15350,00' Thu, 3 Aug 2017 16:

...

1061 Mon, 26 Mar 2018 17:09:18 +0300
!!!error - invalid literal for int() with base 10: '382,00' Tue, 21 Mar 2017 14:59:55 +0300
903 Sun, 8 Apr 2018 19:54:20 +0300
!!!error - invalid literal for int() with base 10: '6020,00' Thu, 4 May 2017 14:12:04 +0300
3694 Mon, 23 Oct 2017 12:24:48 +0300
!!!error - invalid literal for int() with base 10: '664,00' Thu, 17 Aug 2017 11:09:18 +0300
!!!error - invalid literal for int() with base 10: '7800,00' Fri, 4 Mar 2016 13:55:10 +0300
!!!error - invalid literal for int() with base 10: '1400,00' Wed, 18 Jan 2017 11:47:51 +0300
12792 Mon, 31 Dec 2018 02:20:24 +0300
12790 Sun, 30 Dec 2018 20:42:21 +0300
!!!error - invalid literal for int() with base 10: '3750,00' Thu, 3 Mar 2016 18:39:37 +0300
!!!error - invalid literal for int() with base 10: '999,00' Mon, 9 Jan 2017 13:33:19 +0300
553 Sun, 6 May 2018 10:36:55 +0300
!!!error - list index out of range Fri, 6 Apr 2018 14:17:24 +0300
307 Mon, 28 May 2018 11:04:21 +0300
!!!error - invalid literal for int

3714 Fri, 20 Oct 2017 20:07:17 +0300
1216 Fri, 16 Mar 2018 00:12:38 +0300
!!!error - invalid literal for int() with base 10: '20382,00' Fri, 1 Sep 2017 16:58:21 +0300
!!!error - invalid literal for int() with base 10: '№' Mon, 18 May 2015 09:13:08 +0300
!!!error - invalid literal for int() with base 10: '2079,00' Sun, 27 Aug 2017 19:41:43 +0300
!!!error - invalid literal for int() with base 10: '1300,00' Wed, 23 Nov 2016 15:21:40 +0300
!!!error - invalid literal for int() with base 10: '4670,00' Mon, 22 May 2017 10:03:35 +0300
3154 Mon, 27 Nov 2017 11:13:00 +0300
992 Sun, 1 Apr 2018 20:00:51 +0300
2254 Wed, 17 Jan 2018 13:48:24 +0300
!!!error - invalid literal for int() with base 10: '3650,00' Mon, 12 Sep 2016 14:52:04 +0300
13302 Mon, 8 Apr 2019 21:24:23 +0300
!!!error - invalid literal for int() with base 10: '797,98' Sun, 5 Mar 2017 19:56:10 +0300
!!!error - invalid literal for int() with base 10: '1550,00' Sun, 11 Sep 2016 23:23:56 +0300
!!!error - invalid literal for int() with ba

!!!error - invalid literal for int() with base 10: '' Tue, 20 Feb 2018 13:23:51 +0300
!!!error - invalid literal for int() with base 10: '97,00' Mon, 6 Feb 2017 23:20:00 +0300
!!!error - invalid literal for int() with base 10: '9550,00' Thu, 10 Mar 2016 10:13:21 +0300
2698 Wed, 20 Dec 2017 17:47:10 +0300
!!!error - invalid literal for int() with base 10: '3850,00' Mon, 30 Nov 2015 19:16:40 +0300
!!!error - invalid literal for int() with base 10: '2700,00' Thu, 12 Jan 2017 01:10:35 +0300
144 Tue, 12 Jun 2018 12:49:09 +0300
!!!error - invalid literal for int() with base 10: '558,02' Sun, 5 Mar 2017 17:22:06 +0300
12870 Thu, 31 Jan 2019 12:00:36 +0300
!!!error - invalid literal for int() with base 10: '3349,00' Mon, 29 Feb 2016 12:44:05 +0300
3732 Thu, 19 Oct 2017 16:11:14 +0300
!!!error - invalid literal for int() with base 10: '1467,00' Wed, 5 Oct 2016 14:45:31 +0300
!!!error - invalid literal for int() with base 10: '4750,00' Sun, 12 Jun 2016 12:33:46 +0300
!!!error - invalid literal f

1498 Mon, 5 Mar 2018 12:48:35 +0300
!!!error - invalid literal for int() with base 10: '9650,00' Mon, 19 Sep 2016 17:58:57 +0300
2057 Tue, 30 Jan 2018 09:40:21 +0300
!!!error - invalid literal for int() with base 10: '1202,61' Thu, 15 Jun 2017 14:52:56 +0300
!!!error - invalid literal for int() with base 10: '9254,00' Fri, 24 Mar 2017 18:59:50 +0300
!!!error - invalid literal for int() with base 10: '2210,00' Sat, 4 Feb 2017 20:01:24 +0300
12896 Wed, 6 Feb 2019 14:34:30 +0300
937 Thu, 5 Apr 2018 12:11:17 +0300
!!!error - invalid literal for int() with base 10: '5500,00' Fri, 29 Jan 2016 14:59:31 +0300
658 Wed, 25 Apr 2018 11:51:13 +0300
!!!error - invalid literal for int() with base 10: '№' Thu, 11 Jun 2015 13:20:16 +0300
44 Mon, 25 Jun 2018 21:07:30 +0300
13502 Fri, 21 Jun 2019 00:18:52 +0300
3512 Fri, 3 Nov 2017 14:46:40 +0300
!!!error - list index out of range Tue, 5 Dec 2017 07:20:27 +0300
!!!error - invalid literal for int() with base 10: '200,00' Sat, 17 Jun 2017 13:59:45 +0300
!

12298 Tue, 25 Sep 2018 22:28:01 +0300
!!!error - invalid literal for int() with base 10: '5750,00' Tue, 29 Sep 2015 12:50:18 +0300
3695 Mon, 23 Oct 2017 09:22:54 +0300
2214 Fri, 19 Jan 2018 16:22:55 +0300
!!!error - invalid literal for int() with base 10: '649,00' Sat, 15 Jul 2017 13:55:07 +0300
!!!error - invalid literal for int() with base 10: '5150,00' Sun, 27 Dec 2015 22:55:35 +0300
216 Tue, 5 Jun 2018 13:32:05 +0300
!!!error - invalid literal for int() with base 10: '320,00' Mon, 26 Jun 2017 23:05:37 +0300
883 Mon, 9 Apr 2018 13:48:42 +0300
!!!error - invalid literal for int() with base 10: '2499,00' Wed, 31 May 2017 10:42:43 +0300
2029 Wed, 31 Jan 2018 18:29:17 +0300
1222 Thu, 15 Mar 2018 19:20:01 +0300
!!!error - invalid literal for int() with base 10: '420,00' Wed, 26 Apr 2017 20:28:54 +0300
!!!error - invalid literal for int() with base 10: '1590,00' Thu, 30 Mar 2017 10:58:57 +0300
!!!error - list index out of range Thu, 30 Nov 2017 14:56:05 +0300
!!!error - invalid literal fo

In [ ]:

import dbstruct
engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)


#dbstruct.Base.metadata.drop_all(engine)
#dbstruct.Base.metadata.create_all(engine)

engine.dispose()

In [ ]:
engine.dispose()